<a href="https://colab.research.google.com/github/QaziSaim/CASE-STUDIES/blob/main/Mistral7B_FineTune_T.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install -q --no-deps xformers trl peft accelerate bitsandbytes kagglehub


In [10]:
from google.colab import userdata
import os
os.environ['KAGGLE_API_KEY'] = userdata.get('kaggle_api_key')
os.environ['KAGGLE_USERNAME'] = userdata.get('kaggle_username')
os.environ['WANDB_API_KEY'] = userdata.get('WANDB_API')
os.environ['HUGGINGFACE_API_KEY'] = userdata.get('HUGGINGFACE_API')

In [4]:
import kagglehub

# Download latest version
path = kagglehub.model_download("mistral-ai/mistral/pyTorch/7b-instruct-v0.1-hf")

print("Path to model files:", path)


100%|██████████| 116/116 [00:00<00:00, 301kB/s]

  0%|          | 0.00/4.72G [00:00<?, ?B/s]


  0%|          | 0.00/23.4k [00:00<?, ?B/s]

100%|██████████| 72.0/72.0 [00:00<00:00, 304kB/s]
100%|██████████| 23.4k/23.4k [00:00<00:00, 5.29MB/s]



  0%|          | 0.00/9.26G [00:00<?, ?B/s]




100%|██████████| 571/571 [00:00<00:00, 1.58MB/s]



100%|██████████| 1.48k/1.48k [00:00<00:00, 5.72MB/s]





  0%|          | 0.00/1.71M [00:00<?, ?B/s]
  0%|          | 1.00M/4.72G [00:00<29:10, 2.89MB/s]

  0%|          | 1.00M/9.26G [00:00<56:57, 2.91MB/s]


 58%|█████▊    | 1.00M/1.71M [00:00<00:00, 3.03MB/s]
100%|██████████| 1.71M/1.71M [00:00<00:00, 4.58MB/s]


  0%|          | 4.00M/9.26G [00:00<16:08, 10.3MB/s]




  0%|          | 0.00/482k [00:00<?, ?B/s]
  0%|          | 9.00M/4.72G [00:00<03:41, 22.8MB/s]



  0%|          | 10.0M/9.26G [00:00<06:43, 24.6MB/s]



100%|██████████| 1.43k/1.43k [00:00<00:00, 3.12MB/s]

  0%|          | 15.0M/4.72G [00:00<02:40, 31.4MB/s]

  0%|          | 16.0M/9.26G [00:00<05:08, 32.2MB/s]


100%|██████████| 482k/482k [00:00<00:00, 1.82MB/s]


  0%|          | 21.0M/9.26G [00:00<04:26, 37.2MB/s]
  0%|          | 21.0M/4.72G [00:00<02:29, 33.6MB/s]
  1%|          | 28.0M/4.72G [00:00<01:59, 42.1MB/s]

  0%|          | 28.0M/9.26G [00:00<03:53, 42.5MB/s]

  0%|          | 33.0M/9.26G [00:01<03:57, 41.8MB/s]
  1%|          | 33.0M/4.72G [00:01<02:13, 37.8MB/s]

  0%|          | 39.0M/9.26G [00:01<03:40, 44.8MB/s]
  1%|          | 38.0M/4.72G [00:01<02:05, 39.9MB/s]

  0%|          | 44.0M/9.26G [00:01<03:57, 41.6MB/s]
  1%|          | 43.0M/4.72G [00:01<02:25, 34.6MB/s]

  1%|          | 49.0M/9.26G [00:01<03:53, 42.3MB/s]

  1%|          | 56.0M/9.26G [00:01<03:25, 48.2MB/s]
  1%|          | 47.0M/4.72G [00:01<02:45, 30.3MB/s]
  1%|          | 54.0M/4.72G [0

Path to model files: /root/.cache/kagglehub/models/mistral-ai/mistral/pyTorch/7b-instruct-v0.1-hf/1


In [5]:
print(path)

/root/.cache/kagglehub/models/mistral-ai/mistral/pyTorch/7b-instruct-v0.1-hf/1


In [7]:
import torch, wandb

In [11]:
wandb.login(
    key=userdata.get('WANDB_API')
)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [12]:
run = wandb.init(
    project='Fine Tune mistral 7B',
    job_type='training',
    anonymous='allow'
)

In [14]:
base_model = "/root/.cache/kagglehub/models/mistral-ai/mistral/pyTorch/7b-instruct-v0.1-hf/1"
dataset_name = 'mlabonne/guanaco-llama2-1k'
new_model = 'mistral_7b_guanaco'

In [15]:
from datasets import load_dataset
dataset = load_dataset(dataset_name,split='train')
dataset['text'][100]

README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001-9ad84bb9cf65a4(…):   0%|          | 0.00/967k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1000 [00:00<?, ? examples/s]

'<s>[INST] cuanto es 2x2 xD [/INST] La respuesta es 4. </s><s>[INST] puedes demostrarme matematicamente que 2x2 es 4? [/INST] En una multiplicación, el producto es el resultado de sumar un factor tantas veces como indique el otro, es decir, si tenemos una operación v · n = x, entonces x será igual a v sumado n veces o n sumado v veces, por ejemplo, para la multiplicación 3 · 4 podemos sumar "3 + 3 + 3 + 3" o "4 + 4 + 4" y en ambos casos nos daría como resultado 12, para el caso de 2 · 2 al ser iguales los dos factores el producto sería "2 + 2" que es igual a 4 </s>'

In [17]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
from trl import SFTTrainer

In [23]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type = 'nf4',
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=False
)

In [25]:
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config = bnb_config,
    torch_dtype = torch.bfloat16,
    device_map = 'auto',
    trust_remote_code = True
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [26]:
model.config.use_cache = False
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()

In [32]:
tokenizer = AutoTokenizer.from_pretrained(base_model,
                                          trust_remote_code=True,
                                          padding_side='right',

                                          add_eos_token=True,
                                          add_bos_token=True
                                        )
tokenizer.pad_token=tokenizer.eos_token

In [33]:
my_pipeline = pipeline(
    'text-generation',
    model=model,
    tokenizer=tokenizer,
    max_length=1000,
    num_return_sequences=1
)

Device set to use cuda:0


In [59]:
query = "Generate a workflow for flood risk mapping using GeoPandas and OpenStreetMap data."
prompt = f"""
You are a geospatial expert. Given the query, generate a step-by-step geospatial workflow using GeoPandas and OpenStreetMap data. Use Chain-of-Thought reasoning to explain each step clearly. Output the workflow as a JSON object with steps, tools, parameters, and reasoning logs.

Query: {query}

### Chain-of-Thought Reasoning:
1. Identify the task and required data.
2. Break it into sequential geoprocessing steps.
3. Select appropriate tools and parameters.
4. Validate data compatibility (e.g., CRS).
5. Generate JSON workflow.

### Output:
explain the flow in 5 steps

"""

In [60]:
x = my_pipeline(prompt)[0]['generated_text']

In [61]:
x

'\nYou are a geospatial expert. Given the query, generate a step-by-step geospatial workflow using GeoPandas and OpenStreetMap data. Use Chain-of-Thought reasoning to explain each step clearly. Output the workflow as a JSON object with steps, tools, parameters, and reasoning logs.\n\nQuery: Generate a workflow for flood risk mapping using GeoPandas and OpenStreetMap data.\n\n### Chain-of-Thought Reasoning:\n1. Identify the task and required data.\n2. Break it into sequential geoprocessing steps.\n3. Select appropriate tools and parameters.\n4. Validate data compatibility (e.g., CRS).\n5. Generate JSON workflow.\n\n### Output:\nexplain the flow in 5 steps\n\n'